In [1]:
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By 
import time
import re
import pandas as pd


options = webdriver.ChromeOptions()
options.add_argument("headless")

In [2]:
#크롬 브라우저 열기
driver = webdriver.Chrome('C:/chromedriver_win32/chromedriver.exe')
driver.get('https://www.instagram.com/')
p_tag = WebDriverWait(driver, timeout =5).until(EC.presence_of_element_located((By.TAG_NAME, "P")))

time.sleep(3)

#인스타그램 로그인을 위한 계정정보
email='dkdud1871@naver.com'
input_id = driver.find_elements_by_css_selector('input._2hvTZ.pexuQ.zyHYP')[0]
input_id.clear()
input_id.send_keys(email)

password = 'ayoung0112!'
input_pw = driver.find_elements_by_css_selector('input._2hvTZ.pexuQ.zyHYP')[1]
input_pw.clear()
input_pw.send_keys(password)
input_pw.submit()

time.sleep(5)

driver.find_element_by_css_selector('button.sqdOP.yWX7d.y3zKF').click()
time.sleep(3)
driver.find_element_by_css_selector('button.aOOlW.HoLwm').click()
time.sleep(3)


In [3]:
def clean_text(text):
    cleaned_text = re.sub('[^ ㄱ-ㅣ가-힣]+',' ',text)
    cleaned_text = ' '.join(cleaned_text.split())
    return cleaned_text

In [4]:
def insta_searching(word):
    url ='https://www.instagram.com/explore/tags/' + str(word)
    return url

In [5]:
def click_first(driver):
    first =driver.find_element_by_css_selector('div._9AhH0')
    first.click()
    time.sleep(7)
    
time.sleep(10)

In [6]:
tag_list =[]

def get_content(driver):
# 1. 현재 페이지의 HTML 정보 가져오기
    html = driver.page_source
    soup = BeautifulSoup(html, 'lxml')
# 2. 본문내용
    try:
        content = soup.select('div.C4VMK > span')[0].text
    except:
        content = ''
# 3. 본문 내용에서 해시태그 가져오기(정규표현식 활용)
    tags = re.findall('#[A-Za-z0-9가-힣]+', content)
    tag = ''.join(tags).replace("#"," ") # "#" 제거
    tag_data = tag.split()
    for tag_one in tag_data:
        tag_list.append(tag_one)
        
    data = [content, tag_data]
    return data

In [7]:
def next_page(driver):
    next_page = driver.find_element_by_css_selector('body > div._2dDPU.CkGkG > div.EfHg9 > div > div > a._65Bje.coreSpriteRightPaginationArrow')
    next_page.click()
    time.sleep(7)

In [8]:
word = "바른먹거리풀무원"
url = insta_searching(word)

driver.implicitly_wait(9)
driver.get(url)
time.sleep(9)

click_first(driver)

result_list=[]

target = 100
for i in range(target):
    try:
        data = get_content(driver)
        result_list.append(data)
        next_page(driver)
    except:
        time.sleep(9)
        next_page(driver)
    

In [9]:
df = pd.DataFrame(result_list)
df.columns = ['본문','해시태그']
df['본문'] = df['본문'].apply(lambda x : clean_text(x))
df['해시태그'] = df['해시태그'].apply(lambda x : clean_text(' '.join(x)))
df = df.drop_duplicates(subset=['본문','해시태그'])

df

,본문,해시태그
0,풀무원 크리스피팝만두 런칭쇼 번가 일 오후 시 김현우가 식품을 한다 심혈을 기울여 ...,풀무원 크리스피팝만두 번가 신상 먹방유튜버 얼리어답터 세상에서가장빨리 신상 물만두 ...
1,우리 직원들 간식이 떨어지게 무섭게 한가득 풀무원간식도착 했습니다 풀무원모두 맛있는...,풀무원간식도착 풀무원모두 풀무원학교급식 바른먹거리풀무원 바른먹거리동궁찜닭 동궁찜닭경...
2,오늘은뭐해줘야하나 띵동띵동 몇일전에신청해봤는데왔구나 레시피공모전 갖고있는재료로쉽게접...,오늘은뭐해줘야하나 띵동띵동 몇일전에신청해봤는데왔구나 레시피공모전 갖고있는재료로쉽게접...
3,사랑해요 풀무원 우유빛깔 풀무원 바른먹거리풀무원 이벤트 선물 감사히 잘 먹겠습니다 ...,바른먹거리풀무원 풀무원은사랑채식을 두근두근 정면 정비빔면 두부텐더 고단백두부큐브 풀무원
4,집앞 마트에서 한번 사먹어봄라이트누들 콩물 콜라보 이것만 먹으면진짜 오래오래 살거같...,라이트누들 병아리콩 곤약면 콩물 저칼로리 다이어트 건강한맛 바른먹거리 풀무원 너무바...
...,...,...
95,ㅋㅋ ㅎ 일에,
96,,
97,바른먹거리 풀무원 마미유 아이들에게 정말로 안전한 가지 유산균과 유산균 함유 면역미...,바른먹거리 풀무원 마미유 유산균 스톨레밀크 어린이유산균 바른먹거리풀무원 어린이건강식...
98,풀무원입니다 여성의 삶의질을 업그레이드 시켜주는 로젠빈 소개드려요 식물성 여성호르몬...,대상받았어요 여성전문제품 바른먹거리풀무원 풀무원로하스 풀무원건강생활 여자이니까 주부...


In [12]:
df.to_csv("인스타_댓글빼고.csv", header = True, index = False, encoding='utf-8-sig')

In [11]:
#driver.close()